In [2]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime

# --- Google Sheets auth (same as yours) ---
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

# --- Master sheet (index) ---
MASTER_SPREADSHEET_KEY = "1fGDYnO2M-PWqMQTBDO78qbgLC-EZKiJIly68_2D-TEA"
master_spreadsheet = client.open_by_key(MASTER_SPREADSHEET_KEY)
master_sheet = master_spreadsheet.worksheet("Master2")
index = get_as_dataframe(master_sheet, evaluate_formulas=True)
index.dropna(how='all', inplace=True)
index.reset_index(drop=True, inplace=True)

# Filter only active projects
active_projects = index[index["Active Flag"] == 1]

# === SQL Server connection ===
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

# Containers to collect all projects' query results
all_app_logs = []
all_coverage = []
all_attendance = []
all_daywise = []

# Helper to ensure worksheet exists (create if not)
def ensure_worksheet(spreadsheet, title, cols=10, rows=100):
    try:
        return spreadsheet.worksheet(title)
    except Exception:
        return spreadsheet.add_worksheet(title=title, rows=str(rows), cols=str(cols))

# === Process each project ===
for idx, row in active_projects.iterrows():
    project_name = row["Project Name"]
    schema_raw = row.get("SQL Schema Name", "")
    schema = str(schema_raw).strip("[]") if pd.notna(schema_raw) else ""
    sheet_key = '1kVUnxOEfD0280X5RzK8CzvoVDfNgfDuYBGzOSC-T9oM'#changed code

    print(f"\n🚀 Processing Project: {project_name} | Schema: {schema}")

    try:
        sheet = client.open_by_key(sheet_key)
        AppLog_Sheet = sheet.worksheet("Supervisor Log")
        Coverage_Sheet = sheet.worksheet("Coverage")
        Attendence_Sheet = sheet.worksheet("Attendence")
        Daywise_Sheet = sheet.worksheet("Day-Wise Summary")
    except Exception as e:
        print(f"❌ Error opening Google Sheet for {project_name}: {e}")
        continue

    try:
        with pyodbc.connect(conn_str) as conn:
            # --- AppLog Query ---
            app_query = f'''
                SELECT a.[SupervisorAppLogId],
                       a.[ManagerId] AS [Supervisor ID],
                       a.[ManagerName] AS [Supervisor Name],
                       b.[UserName], 
                       a.[UserFullName], 
                       a.[UserDesignation], 
                       a.[LISStoreCode], 
                       a.[StoreName],
                       a.[ModuleName], 
                       a.[Count],
                       a.[Time],
                       CAST(a.[Time] AS DATE) AS LogDate 
                FROM [{database}].[{schema}].[SupervisorAppLogReport] a
                LEFT JOIN [{database}].[{schema}].[UserMaster] b
                ON a.[UserId]=b.[UserId]
                WHERE MONTH([Time]) = MONTH(GETDATE() - 1)
                  AND YEAR([Time]) = YEAR(GETDATE() - 1)
            '''
            AppLog = pd.read_sql(app_query, conn)
            # attach meta
            AppLog["ProjectName"] = project_name
            AppLog["Schema"] = schema
            all_app_logs.append(AppLog)

            # AppLog_Sheet.batch_clear(["A:M"])
            # set_with_dataframe(AppLog_Sheet, AppLog)
            print("✅ Uploaded AppLog (per-project)")

            # --- Coverage Query ---
            coverage_query = f'''
                SELECT ISPStoreAttendance_NormalExportId,
                       SupervisorCode,
                       ISPCode,
                       UserName,
                       FullName,
                       Designation,
                       ClientStoreCode,
                       LISStoreCode,
                       StoreName,
                       StoreAddress,
                       Format,
                       ChainCode, 
                       ChainName,
                       VisitDate,
                       PJPStatus,
                       [CheckIn-Time],
                       [CheckOut-Time],
                       TimeDuration, 
                       AutoCheckOutStatus,
                       NoPermissionReason,
                       AttendanceType, 
                       IsAudited,
                       TRY_CONVERT(DATE, VisitDate, 105) AS DateOnly,
                       CONCAT(CONCAT(TRY_CONVERT(DATE, VisitDate, 105), UserName), LISStoreCode) AS Key1
                FROM [{database}].[{schema}].[ISPStoreAttendance_NormalExport]
                WHERE MONTH(PARSE(VisitDate AS datetime USING 'en-GB')) = MONTH(GETDATE() - 1)
                  AND YEAR = YEAR(GETDATE() - 1)
                  AND (UserName LIKE '%SUP%'
                  OR UserName LIKE '%MCR-TL%')
            '''
            Coverage = pd.read_sql(coverage_query, conn)
            Coverage["ProjectName"] = project_name
            Coverage["Schema"] = schema
            all_coverage.append(Coverage)

            # Coverage_Sheet.batch_clear(["A:X"])
            # set_with_dataframe(Coverage_Sheet, Coverage)
            print("✅ Uploaded Coverage (per-project)")

            # --- Attendance Query ---
            attendance_query = f'''
                SELECT * FROM (
                    SELECT ISPAttendanceMasterExportId,
                           SupervisorCode, 
                           ISPCode,
                           EmployeeId, 
                           UserName,
                           FullName,
                           Designation,
                           FORMAT(TRY_CONVERT(date, [Date], 105), 'dd-MM-yyyy') as Date, 
                           VisitDate,
                           [In-Time], 
                           [Out-Time], 
                           Status,
                           ApprovalStatus,
                           ApprovalDate,
                           ApprovedBy,
                           Rejectedreason,
                           CoveredStatus,
                           MerchandisingStatus,
                           EmployeeStatus,
                           ROW_NUMBER() OVER(PARTITION BY UserName, 
                           FullName,
                           Date ORDER BY [In-Time] DESC) AS Rank,
                           CASE WHEN UserName LIKE '%SUP%' THEN 1
                                WHEN UserName LIKE '%MCR-TL%' THEN 1
                                ELSE 0 END AS SupFlag
                    FROM [{database}].[{schema}].[ISPAttendanceMasterExport]
                    WHERE [Month] = MONTH(GETDATE() - 1)
                      AND [Year] = YEAR(GETDATE() - 1)
                      AND UserName NOT LIKE '%test%'
                      AND UserName NOT LIKE '%Test%'
                ) x WHERE Rank = 1
            '''
            Attendence = pd.read_sql(attendance_query, conn)
            Attendence.drop(columns=['Rank'], inplace=True, errors='ignore')
            Attendence["ProjectName"] = project_name
            Attendence["Schema"] = schema
            all_attendance.append(Attendence)

            # update LastUpdated column row 0 if exists — keep behavior but safe guard
            try:
                Attendence.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            except Exception:
                # if DataFrame empty or column missing, ignore
                pass

            # Attendence_Sheet.batch_clear(["A:U"])
            # set_with_dataframe(Attendence_Sheet, Attendence)
            print("✅ Uploaded Attendance (per-project)")

            # --- Day-Wise Summary (per project) ---
            filtered_df = Attendence[Attendence["SupFlag"] == 1].copy() if "SupFlag" in Attendence.columns else Attendence.copy()
            filtered_df["Date"] = pd.to_datetime(filtered_df.get("Date", pd.Series([])), dayfirst=True, errors="coerce").dt.strftime('%d-%m-%Y')
            Day_Wise_Summary = filtered_df[["UserName", "FullName", "Date"]].drop_duplicates().reset_index(drop=True)

            # Reformat Coverage DateOnly and compute aggregated coverage as in your original script
            Coverage["DateOnly"] = pd.to_datetime(Coverage["DateOnly"], dayfirst=True, errors='coerce').dt.strftime('%d-%m-%Y')
            Coverage["TimeDuration"] = Coverage["TimeDuration"].astype(str).fillna("00:00")
            Coverage["TimeDuration"] = Coverage["TimeDuration"].replace(r'[^0-9:]', '00:00', regex=True)
            Coverage["TimeDuration"] = Coverage["TimeDuration"].apply(lambda x: x if len(x.split(':')) == 3 else x + ":00")
            Coverage["TimeDuration"] = pd.to_timedelta(Coverage["TimeDuration"], errors='coerce').fillna(pd.Timedelta(seconds=0))

            agg_coverage = Coverage.groupby(["UserName", "DateOnly"]).agg({
                "CheckIn-Time": "min",
                "CheckOut-Time": "max",
                "TimeDuration": "sum",
                "LISStoreCode": pd.Series.nunique
            }).reset_index()

            agg_coverage.rename(columns={
                "DateOnly": "Date",
                "CheckIn-Time": "First Store Login",
                "CheckOut-Time": "Last Store Logout",
                "TimeDuration": "CFT",
                "LISStoreCode": "Outlets Covered"
            }, inplace=True)

            agg_coverage["CFT"] = agg_coverage["CFT"].apply(
                lambda x: f"{int(x.total_seconds() // 3600):02d}:{int((x.total_seconds() % 3600) // 60):02d}" if pd.notna(x) else "00:00"
            )

            Day_Wise_Summary = Day_Wise_Summary.merge(agg_coverage, on=["UserName", "Date"], how="left")
            Day_Wise_Summary = Day_Wise_Summary.fillna("-")

            def to_time_obj(t):
                try:
                    if t == "-" or pd.isnull(t):
                        return pd.NaT
                    return pd.to_datetime(t, format="%H:%M:%S", errors='coerce')
                except:
                    return pd.NaT

            Day_Wise_Summary["FirstTimeParsed"] = Day_Wise_Summary["First Store Login"].apply(to_time_obj)
            Day_Wise_Summary["LastTimeParsed"] = Day_Wise_Summary["Last Store Logout"].apply(to_time_obj)

            def calc_fos(row):
                start = row["FirstTimeParsed"]
                end = row["LastTimeParsed"]
                first_login = row["First Store Login"]
                if first_login == "-" or pd.isnull(start) or pd.isnull(end):
                    return "-"
                diff = end - start
                if diff.total_seconds() >= 0:
                    minutes = int(diff.total_seconds() // 60)
                    return f"{minutes // 60:02d}:{minutes % 60:02d}"
                return "00:00"

            Day_Wise_Summary["FOS"] = Day_Wise_Summary.apply(calc_fos, axis=1)
            Day_Wise_Summary.drop(columns=["FirstTimeParsed", "LastTimeParsed"], inplace=True, errors='ignore')

            desired_order = ["UserName", "FullName", "Date", "First Store Login",
                             "Last Store Logout", "CFT", "FOS", "Outlets Covered"]
            # ensure columns exist
            for col in desired_order:
                if col not in Day_Wise_Summary.columns:
                    Day_Wise_Summary[col] = "-"

            Day_Wise_Summary = Day_Wise_Summary[desired_order]
            Day_Wise_Summary = Day_Wise_Summary.sort_values("Date").reset_index(drop=True)

            # attach meta and append to list
            Day_Wise_Summary["ProjectName"] = project_name
            Day_Wise_Summary["Schema"] = schema
            all_daywise.append(Day_Wise_Summary)

            # Daywise_Sheet.batch_clear(["A:H"])
            # set_with_dataframe(Daywise_Sheet, Day_Wise_Summary)
            print("✅ Uploaded Day Wise Summary (per-project)")

            # Update Last Run Time in master index
            now_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            index.at[idx, "Last Run Time"] = now_str
            print(f"🕒 Updated Last Run Time for {project_name}: {now_str}")

    except Exception as e:
        print(f"❌ Error processing {project_name}: {e}")

# === After loop: concatenate and write consolidated sheets to master spreadsheet ===
def concat_and_write(df_list, spreadsheet, sheet_title):
    if not df_list:
        print(f"⚠️ No data for {sheet_title}")
        # ensure worksheet cleared
        try:
            ws = ensure_worksheet(spreadsheet, sheet_title)
            ws.clear()
        except Exception:
            pass
        return
    combined = pd.concat(df_list, ignore_index=True, sort=False)
    # optional: drop duplicates if any (comment/uncomment based on preference)
    combined = combined.drop_duplicates().reset_index(drop=True)
    # write to spreadsheet
    ws = ensure_worksheet(spreadsheet, sheet_title, rows=max(100, len(combined)+10), cols= max(10, combined.shape[1]+2))
    ws.batch_clear(["A:Z"])  # clear a broad range
    set_with_dataframe(ws, combined)
    print(f"✅ Wrote consolidated '{sheet_title}' with {len(combined)} rows.")

# Write consolidated sheets
concat_and_write(all_app_logs, master_spreadsheet, "All_AppLog")
concat_and_write(all_coverage, master_spreadsheet, "All_Coverage")
concat_and_write(all_attendance, master_spreadsheet, "All_Attendance")
concat_and_write(all_daywise, master_spreadsheet, "All_Daywise_Summary")

# === Write Updated Index (Master) Sheet Back ===
# master_sheet.clear()
# set_with_dataframe(master_sheet, index)
# print("\n✅ Master sheet updated with Last Run Times.")



🚀 Processing Project: Piramal | Schema: PiramalManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AppLog = pd.read_sql(app_query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Coverage = pd.read_sql(coverage_query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence = pd.read_sql(attendance_query, conn)


✅ Uploaded AppLog (per-project)
✅ Uploaded Coverage (per-project)
✅ Uploaded Attendance (per-project)
✅ Uploaded Day Wise Summary (per-project)
🕒 Updated Last Run Time for Piramal: 2025-10-14 10:57:32

🚀 Processing Project: ZydusGT | Schema: ZydusGTManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  AppLog = pd.read_sql(app_query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Coverage = pd.read_sql(coverage_query, conn)


✅ Uploaded AppLog (per-project)
✅ Uploaded Coverage (per-project)
✅ Uploaded Attendance (per-project)
✅ Uploaded Day Wise Summary (per-project)
🕒 Updated Last Run Time for ZydusGT: 2025-10-14 10:57:37


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_22064\457252379.py:180: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Attendence = pd.read_sql(attendance_query, conn)


✅ Wrote consolidated 'All_AppLog' with 894 rows.
✅ Wrote consolidated 'All_Coverage' with 501 rows.
✅ Wrote consolidated 'All_Attendance' with 2483 rows.
✅ Wrote consolidated 'All_Daywise_Summary' with 170 rows.
